In [76]:
import numpy
import pandas # Obsługa CSV
import random
import matplotlib.pyplot as plt # Wykresy
import seaborn # Automatyczne wykresy
seaborn.set_palette('husl') # Paleta barw wykresów

baza = pandas.read_csv("FlowerFamily.csv")
baza

,type,length[cm],width[cm],thickness[mm],fraying(0-1),hairiness(0-1),leafdensity(leaf/stalk),stalkcircumference[mm],height[cm],stalkquantity
0,Araceae,47.9,17.4,0.1,0.8,0.0,1,19.1,37.5,4
1,Piperaceae,1.8,3.9,0.7,0.0,0.4,17,2.2,1.9,10
2,Crassulaceae,4.3,4.3,1.4,0.0,0.4,11,23.5,22.3,2
3,Piperaceae,2.7,1.0,0.7,0.0,0.2,7,3.5,2.0,7
4,Violaceae,32.9,13.4,1.2,0.6,0.5,1,20.7,4.8,1
...,...,...,...,...,...,...,...,...,...,...
643,Arecaceae,21.5,2.0,0.2,0.0,0.0,1,19.7,109.6,2
644,Arecaceae,57.9,2.2,0.2,0.0,0.0,1,18.4,70.2,4
645,Violaceae,30.1,28.8,1.5,0.4,0.5,1,20.6,7.4,1
646,Crassulaceae,5.2,3.4,1.0,0.0,0.6,12,19.1,23.7,4


In [77]:
class DataProcessing:
    # Tasowanie
    @staticmethod
    def shuffle(zbior):
        for i in range(len(zbior)-1,0,-1):
            j = random.randint(0,i+1)
            zbior.iloc[i], zbior.iloc[j] = zbior.iloc[j], zbior.iloc[i]
        return zbior
    
    #Podział zbioru
    @staticmethod
    def splitSet(zbior):
        border = int((len(zbior)*50)/100)
        train = zbior.iloc[:border]
        val = zbior.iloc[border:]
        return train, val

In [78]:
class SoftSet:

    @staticmethod
    def softening(X, sample, a, b, n):
        X_, Y_ = SoftSet.simplify(X, a, b, n)
        fittingNames = SoftSet.classify(sample, X_, Y_)
        if sample != [1,1]:
            fittingNames.remove('Violaceae')
        
        fittingDataFrame = SoftSet.dfSlice(X,fittingNames,n)
        return fittingNames, fittingDataFrame
        
         
    @staticmethod
    def dfSlice(X,fittingNames,n):
        sliced = X.loc[X['type'].isin(fittingNames)] 
        return sliced
    
    #Simplification of DataFrame by column 4 and 5
    @staticmethod
    def simplify(X, a, b, n):
        
        #Creating list of types
        list(X['type'])
        specNames = []
        types = list(train['type'])

        for i in types:
            if i not in specNames:
                specNames.append(i)
        #End of creating list of types
        
        #Creating list of lists called 'species' for each type of plant
        species = []
        
        #Sorting X by types
        X.sort_values(by='type', inplace=True)

        #Filling 'species' list with records
        j = 0
        k = 0
        
        for i in range(len(X)): #before: len(X)-1
            name = specNames[j]
            if X.iloc[i,n] != name:
                species.append(pandas.DataFrame(X.iloc[k:i]))
                k = i
                j += 1
        species.append(pandas.DataFrame(X.iloc[k:i]))
        #End of filling 'species' list
        
        #Creating list of column names from given dataframe
        colNames = list(X.columns)
        
        #Creating table called 'softTable' for storing certain type softset specification
        softTable = []
        
        for i in range(0,len(specNames)):
            softTable.append(dict())
                                #fraying(0-1) hairiness(0-1)
            softTable[i].update({colNames[4]:0,colNames[5]:0})

        #End of creating table 'softTable'        
        #Filling 'softTable'
        for k in range(0,2,1):
            avr = 0
            
            for i in range(0,len(specNames),1):
                #print("i: "+str(i))
                avrS = 0
                for j in range(0,len(species[i]),1):
                    avrS += species[i].iloc[j,k+4]
                       
                avrS = avrS/(len(species[i])-1)

                if avrS > avr:
                        softTable[i][colNames[k+4]] = 1
                        
        #End of filling 'softTable'
        return softTable, specNames
    
    
#Classification of a softset, return list of not eliminated types
    @staticmethod
    def classify(sample, Y, X):
        Z = []
        for i in range(0,len(X),1):
            z = 0
            #print(i)
            key = list(Y[0].keys())
            
            for j in range(0,len(Y[i]),1):
                z += sample[j]*Y[i][key[j]]
            Z.append(z)
        
        #finding the fitting ones by determinating if they do have certain characteristics
        #if Z[i] has the or has not the specific characteristic, then it 'fits'
        Fit = []
        fitting = max(Z)
        
        for i in range(len(X)):
            if Z[i]>=fitting:
                Fit.append(X[i])
                
        return Fit


In [79]:
class naiveBayes:
    #średnia
    @staticmethod
    def srednia(atr):
        return sum(atr)/len(atr)
    
    #odchylenie standardowe
    @staticmethod
    def odchylenie(atr):
        suma = 0
        sr = naiveBayes.srednia(atr)
        for element in atr:
            suma += (element - sr)**2
        return numpy.sqrt(suma/len(atr))
    
    #gauss
    @staticmethod
    def gauss(at1, mean, std):
        exponent = numpy.exp(-(at1-mean)**2/(2*std**2))
        return 1/numpy.sqrt(2*numpy.pi*std**2)*exponent
    
    #klasyfikacja
    def classify(sample, trainSet):
        setsort = trainSet.groupby(trainSet.columns[0])
        sr = setsort.mean()
        odch = setsort.std()
        atrs = list(sr)
        names = list(setsort.groups.keys())
        maxi = -1
        ind = -1
        for name in names:
            temp_prob = 1/len(names)
            for atr in atrs:
                temp_prob *= naiveBayes.gauss(sample[atrs.index(atr)],sr[atr][name],odch[atr][name])
            if maxi < temp_prob:
                maxi = temp_prob
                ind = names.index(name)
        return names[ind]

In [80]:
class Integration:
    @staticmethod
    def classify(sample, trainSet):
        softsample = [1,1]
        if sample['fraying(0-1)'] == 0:
            softsample[0] = 0
        if sample['hairiness(0-1)'] == 0:
            softsample[1] = 0
        fittingNames, fittingDataFrame = SoftSet.softening(trainSet, softsample, 1, len(trainSet), 0)
#         print(fittingNames, end=" | ")
        if len(fittingNames) == 1:
#             print("pewnik", end=" | ")
            return fittingNames[0]
        else:
            return naiveBayes.classify(sample, fittingDataFrame)


In [81]:
train, val = DataProcessing.splitSet(DataProcessing.shuffle(baza))

In [83]:
correct = 0
for i in range(len(val)):
    sample = (val.iloc[i][1:])
    wynik = Integration.classify(sample, train)
    if wynik == val.iloc[i]['type']:
        correct += 1

print("Integration accuracy is", correct/len(val)*100, '%')

correct = 0
for i in range(len(val)):
    sample = (val.iloc[i][1:])
    wynik = naiveBayes.classify(sample, train)
    if wynik == val.iloc[i]['type']:
        correct += 1

print("Bayes-only accuracy is", correct/len(val)*100, '%')

<ipython-input-78-3e71599c84a1>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.sort_values(by='type', inplace=True)
<ipython-input-79-1484bc3cef7a>:19: RuntimeWarning: invalid value encountered in double_scalars
  exponent = numpy.exp(-(at1-mean)**2/(2*std**2))
<ipython-input-79-1484bc3cef7a>:20: RuntimeWarning: divide by zero encountered in double_scalars
  return 1/numpy.sqrt(2*numpy.pi*std**2)*exponent
<ipython-input-79-1484bc3cef7a>:19: RuntimeWarning: divide by zero encountered in double_scalars
  exponent = numpy.exp(-(at1-mean)**2/(2*std**2))
<ipython-input-79-1484bc3cef7a>:20: RuntimeWarning: invalid value encountered in double_scalars
  return 1/numpy.sqrt(2*numpy.pi*std**2)*exponent


Integration accuracy is 52.77777777777778 %
Bayes-only accuracy is 17.59259259259259 %
